# Feature Engineering

In [1]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import VectorAssembler, MinMaxScaler
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, ArrayType
from pyspark.sql.functions import col, when, size, expr, udf

import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

spark = SparkSession.builder.appName("Forum Question Analyzer") \
    .config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.17.0") \
    .getOrCreate()
posts = spark.read \
    .format("com.databricks.spark.xml") \
    .option("rootTag", "posts") \
    .option("rowTag", "row") \
    .load("tex.stackexchange.com/Posts.xml") \
    .alias('posts')
users = spark.read \
    .format("com.databricks.spark.xml") \
    .option("rootTag", "users") \
    .option("rowTag", "row") \
    .load("tex.stackexchange.com/Users.xml") \
    .alias('users')
tags = spark.read \
    .format("com.databricks.spark.xml") \
    .option("rootTag", "tags") \
    .option("rowTag", "row") \
    .load("tex.stackexchange.com/Tags.xml") \
    .alias('tags')

## Data preparation

In [6]:
posts = posts.filter(posts._PostTypeId==1)
questions = posts.withColumn("_Tags", expr("split(substring(_Tags, 2, length(_Tags) - 2), '><')"))\
            .withColumn("_Body", size(expr("split(_Body, ' ')")))\
            .withColumn("_Title", size(expr("split(_Title, ' ')")))

questions = questions.join(users, questions._OwnerUserId == users._Id).select(
    questions._Id.alias("QuestionId"),
    questions._Body.alias("BodyLength"),
    questions._Title.alias("TitleLength"),
    questions._Tags.alias("TagsCounts"),
    size(questions._Tags).alias("NumberOfTags"),
    users._Id.alias("OwnerId"),
    users._DownVotes.alias("OwnerDownVotes"),
    users._UpVotes.alias("OwnerUpVotes"),
    users._Reputation.alias("OwnerReputation"),
    users._Views.alias("OwnerViews"),
    (questions._CreationDate - users._CreationDate).cast("integer").alias("UserExperience"),
    when(col("_AcceptedAnswerId").isNull(), 0).otherwise(1).alias("Accepted")
)

#Fast way to change tag names to tag counts from Tags table
#####
tag_counts = tags.select("_TagName", "_Count").rdd.collectAsMap()

def replace_tags_with_counts(tags):
    return [tag_counts.get(tag, 0) for tag in tags]

replace_tags_with_counts_udf = udf(replace_tags_with_counts, ArrayType(IntegerType()))
#####

data = questions.withColumn("TagsCounts", replace_tags_with_counts_udf(questions.TagsCounts))

## Feature importances

In [17]:
#features = ['_Score', '_ViewCount', '_AnswerCount', '_CommentCount']
features = ['BodyLength', 'TitleLength', 'NumberOfTags', 'OwnerDownVotes', 'OwnerUpVotes', 'OwnerReputation', 'OwnerViews', 'UserExperience']
assembler = VectorAssembler(inputCols=features, outputCol="features")
scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")

In [18]:
rf = RandomForestClassifier(labelCol="Accepted", featuresCol="scaledFeatures", numTrees=10)
rf_pipeline = Pipeline(stages=[assembler, scaler, rf])
rf_model = rf_pipeline.fit(data)
result = rf_model.stages[-1].featureImportances.toArray()

In [19]:
print("Feature Importances:")
for feature, importance in zip(features, sorted(result, reverse=True)):
    print(f"{feature}: {importance}")

Feature Importances:
BodyLength: 0.5506458833729864
TitleLength: 0.3276630099522496
NumberOfTags: 0.10554208561961907
OwnerDownVotes: 0.009440315823911916
OwnerUpVotes: 0.004258999288319472
OwnerReputation: 0.0017422204530491048
OwnerViews: 0.0005202815276751052
UserExperience: 0.00018720396218923442
